In [ ]:
%%capture
!pip install catboost

In [ ]:
%cd '/content/drive/MyDrive/mchs/emergency_datahack'

from notebooks.notebook_utils import add_project_path
add_project_path()
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import datetime
import features as f 
from match_station import coord_merge, stations_list, coord_km
from optimizers import optimize
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

SEED = 22

/content/drive/MyDrive/mchs/emergency_datahack
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
test = pd.read_csv('./data_raw/test.csv', parse_dates=['datetime'])

In [66]:
train = pd.read_pickle('./data_processed/train2020_final_2.pickle.gz')

In [67]:
test = train[train.datetime.isin(test.datetime)]
train = train[~train.datetime.isin(test.datetime)]

In [68]:
cat_features = [
                'road_id', 
                'lane_count', 'vsp_1_q', 'vsp_2_q', 'vsp_3_q', 
                'visib_q', 'clouds', 'clouds_q', 'weather_range', 'weather_range_q',
                'weather_on_measure', 'weather_on_measure_q', 'wind_dir_q',
                'avg_wind_q', 'max_wind_q', 'precip_q', 'temp_on_measure_q', 
                'temp_min_q', 'temp_max_q', 'humidity_q', 'pressure_q', 
                'weather_range_diff', 'weather_on_measure_diff', 'weekday', 'month', 'hour'
                ]

In [69]:
for f in cat_features:
  test[f].fillna(-1, inplace=True)
  test[f] = test[f].astype(int)

In [70]:
test_pool = Pool(test.drop(['target', 'datetime'], axis=1), 
                 cat_features=cat_features)

In [96]:
from_file = CatBoostClassifier()

from_file.load_model('/content/drive/MyDrive/mchs/emergency_datahack/results/cb_model_best.pt')

In [97]:
from_file.get_params()

{'class_weights': [0.01, 1, 2],
 'depth': 8,
 'eval_metric': 'TotalF1:weighted=false;average=Macro',
 'iterations': 2000,
 'learning_rate': 0.2,
 'loss_function': 'MultiClass',
 'min_data_in_leaf': 1,
 'od_type': 'Iter',
 'od_wait': 30,
 'one_hot_max_size': 2,
 'random_seed': 22,
 'task_type': 'GPU',
 'use_best_model': True,
 'verbose': 10}

In [98]:
predictions = from_file.predict(test_pool, 'Class')
# pred_probas = from_file.predict(test_pool, 'Probability')


In [99]:
subm_labels = test[['datetime','road_id','road_km','target']]
subm_labels['target'] = predictions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [100]:
subm_labels.target.value_counts()

0    1744959
1       1929
2          4
Name: target, dtype: int64

In [94]:
submission = pd.read_csv('./data_raw/test.csv', parse_dates=['datetime'])
submission = pd.merge(submission[['datetime','road_id','road_km']], subm_labels[['datetime','road_id','road_km', 'target']], on=['datetime','road_id','road_km'])
submission = submission[submission.target!=0].reset_index(drop=True)
submission.to_csv('./results/submission_2021-05-30 04:54:07.000645.csv', index=False)

In [95]:
submission.target.value_counts()

1    1929
2       4
Name: target, dtype: int64